In [1]:
import torch
from copy import copy

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [3]:
def preprocess_function(examples):
    # Repeat each prompt for 5 times to go with the 5 possibilities of each option
    first_sentences = [[context] * 2 for context in examples["startphrase"]]
    # Grab all options
    second_sentences = [[ending1, examples['ending2'][i]] for i, ending1 in enumerate(examples['ending1'])]
#     print(first_sentences)
#     print("====")

    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

#     print(first_sentences)
#     print("====")
#     print(second_sentences)

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {
        k: [v[i : i + 2] for i in range(0, len(v), 2)]
        for k, v in tokenized_examples.items()
    }

In [4]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import os
from math import inf

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: AutoTokenizer
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [5]:
from datasets import load_dataset

In [6]:
# ds = load_dataset("super_glue", "copa", cache_dir="/scratch/afz225/.cache")
ds = load_dataset("nightingal3/fig-qa")

In [7]:
columns = copy(ds['train'].column_names)
# columns.remove('label')
columns.remove('labels')
ds = ds.map(preprocess_function, batched=True, remove_columns=columns)

Map:   0%|          | 0/9674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

Map:   0%|          | 0/1146 [00:00<?, ? examples/s]

In [8]:
ds['train']

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 9674
})

In [9]:
# ds = ds.rename_column(original_column_name="label", new_column_name="labels")

In [10]:
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [12]:
from transformers import AutoModelForMultipleChoice
from adapters import init,AutoAdapterModel

# model = AutoModelForMultipleChoice.from_pretrained("roberta-base", cache_dir="/scratch/afz225/.cache")
model = AutoModelForMultipleChoice.from_pretrained("FacebookAI/xlm-roberta-base")
# Enable adapter support
init(model) 

Some weights of XLMRobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# model.load_adapter("./col-adapter")

OSError: Unable to load adapter ./col-adapter from any source. Please check the name of the adapter or the source.

In [14]:
model

XLMRobertaForMultipleChoice(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttentionWithAdapters(
              (query): LoRALinearTorch(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (key): LoRALinearTorch(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (value): LoRALinearTorch(
                in_features=768, out_features=768, 

In [15]:
dialects = ["std-dia", "aus", "hon", "nig", "col", "wel"]

In [16]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

2024-04-27 11:49:10.132531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 11:49:10.132895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 11:49:10.211493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-27 11:49:10.355007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 11:49:11.962170: W tensorflow/compiler/tf2

In [17]:
def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

In [18]:
dialect = 'aus'

In [19]:
for dialect in dialects:
    if dialect != "std-dia":
        # ds = load_dataset("ashabrawy/dia_copa", dialect, cache_dir="/scratch/afz225/.cache")
        ds = load_dataset("ashabrawy/dia_figqa", dialect)
        columns = copy(ds['train'].column_names)
        # columns.remove('label')
        columns.remove('labels')
        ds = ds.map(preprocess_function, batched=True, remove_columns=columns)
        # ds = ds.rename_column(original_column_name="label", new_column_name="labels")
        # ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    model.add_adapter(dialect, config="seq_bn")
    model.train_adapter(dialect)
    training_args = TrainingArguments(
        learning_rate=1e-4,
        num_train_epochs=6,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        logging_steps=200,
        output_dir=f"/l/users/abdelrahman.sadallah/dialectal_gen-{dialect}",
        overwrite_output_dir=True,
        # The next line is important to ensure the dataset labels are properly passed to the model
        remove_unused_columns=False,
    )
    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["validation"],
        compute_metrics=compute_accuracy,
        data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    )
    trainer.train()
    model.save_adapter(f"/l/users/abdelrahman.sadallah/dialectal_gen/{dialect}-adapter-figqa-xlmr", dialect, with_head=True)
    

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bodasadallah2. Use `wandb login --relogin` to force relogin


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
200,0.694100
400,0.695600
600,0.694700
800,0.693700
1000,0.693100
1200,0.694300
1400,0.693900
1600,0.694700
1800,0.693300


Map:   0%|          | 0/9674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1146 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/9674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1146 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
200,0.694900
400,0.695000
600,0.694200
800,0.693800
1000,0.693600
1200,0.694000
1400,0.693500
1600,0.693700
1800,0.694100


In [20]:
model.average_adapter("avg", dialects)

In [21]:
model.active_adapters = "avg"

In [22]:
import pandas as pd

In [23]:
def dialect_eval(model):
    dialects = ["std-dia", "aus", "nig", "wel", "col", "hon"]
    for dialect in dialects:
        if dialect == "std-dia":
            # eval_ds = load_dataset("super_glue", "copa", cache_dir="/scratch/afz225/.cache")['validation']
            eval_ds = load_dataset("nightingal3/fig-qa", cache_dir="/scratch/afz225/.cache")['validation']
            columns = copy(eval_ds.column_names)
            # columns.remove('label')
            columns.remove('labels')
            eval_ds = eval_ds.map(preprocess_function, batched=True, remove_columns=columns)
            # eval_ds = eval_ds.rename_column(original_column_name="label", new_column_name="labels")
        else:
            # eval_ds = load_dataset("ashabrawy/dia_copa", dialect, cache_dir="/scratch/afz225/.cache")['validation']
            eval_ds = load_dataset("ashabrawy/dia_figqa", dialect, cache_dir="/scratch/afz225/.cache")['validation']
            columns = copy(eval_ds.column_names)
            # columns.remove('label')
            columns.remove('labels')
            eval_ds = eval_ds.map(preprocess_function, batched=True, remove_columns=columns)
            # eval_ds = eval_ds.rename_column(original_column_name="label", new_column_name="labels")
        training_args = TrainingArguments(
            learning_rate=1e-4,
            num_train_epochs=6,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            logging_steps=200,
            output_dir=f"./training_output-{dialect}",
            overwrite_output_dir=True,
            # The next line is important to ensure the dataset labels are properly passed to the model
            remove_unused_columns=False,
        )
        trainer = AdapterTrainer(
            model=model,
            args=training_args,
            train_dataset=eval_ds,
            eval_dataset=eval_ds,
            compute_metrics=compute_accuracy,
            data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
        )
        eval_metrics = trainer.evaluate(eval_ds)
        eval_metrics['test_set'] = dialect
        
        results = pd.DataFrame(eval_metrics, index=[0]) if dialect == "std-dia" else pd.concat([results, pd.DataFrame(eval_metrics, index=[0])], ignore_index=True)
    return results

In [24]:
dialect_eval(model).to_csv("avg-adapter-figqa-xlmr.csv")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:
for dialect in dialects:
    model.active_adapters = dialect
    results = dialect_eval(model)
    print(results)
    results.to_csv(f"{dialect}-adapter-figqa-xlmr.csv")

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.204, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


   eval_loss  eval_acc  eval_runtime  eval_samples_per_second  \
0   0.693128  0.499086        1.2858                  850.846   
1   0.693114  0.507313        1.3573                  805.985   
2   0.693138  0.517367        1.2361                  885.038   
3   0.693100  0.522852        1.3597                  804.572   
4   0.693126  0.528336        1.3816                  791.833   
5   0.693154  0.495430        1.2627                  866.366   

   eval_steps_per_second test_set  
0                 27.221  std-dia  
1                 25.786      aus  
2                 28.315      nig  
3                 25.740      wel  
4                 25.333      col  
5                 27.717      hon  


wandb: Network error (ReadTimeout), entering retry loop.
